In [ ]:
# Kaggle에서는 필요없음 
# !pip install torch 
# !pip install torchvision

### 패션 MNIST
- 구두, 가방, 옷 등의 이미지로 구성
- 총 10개의 class
- 28 * 28 pixel = 784개의 Feature
- Sample수는 70000개 이다. 

In [1]:
# 데이터 불러오기
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# 학습 데이터 불러오기
train_data = datasets.FashionMNIST(
                root = "./Data", # 데이터가 저장될 경로 
                train = True, # True이면 학습데이터, False이면 테스트 데이터 
                download = True, # True이면 데이터가 없을 경우 자동 다운로드 
                transform = ToTensor() # 이미지를 텐서로 변환 
)

100%|██████████| 26.4M/26.4M [00:00<00:00, 116MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 3.94MB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 60.8MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 10.0MB/s]


In [3]:
# 테스트 데이터 불러오기
test_data = datasets.FashionMNIST(
                root = "./Data", # 데이터가 저장될 경로 
                train = False, # True이면 학습데이터, False이면 테스트 데이터 
                download = True, # True이면 데이터가 없을 경우 자동 다운로드 
                transform = ToTensor() # 이미지를 텐서로 변환 
)